In [1]:
!pwd

/models/research/object_detection


### Generate data

Generate training data to `./images/`

In [2]:
!python xml_to_csv.py
print("")
!ls images -lh
print("")
!head images/train_labels.csv

Successfully converted xml to csv.
Successfully converted xml to csv.

total 56K
drwxr-xr-x 2 root root 4.0K Oct 13 02:15 test
-rw-r--r-- 1 root root 5.9K Nov  3 18:38 test_labels.csv
drwxr-xr-x 2 root root  20K Oct 13 02:15 train
-rw-r--r-- 1 root root  22K Nov  3 18:38 train_labels.csv

filename,width,height,class,xmin,ymin,xmax,ymax
cam_image28.jpg,960,540,ten,268,115,502,426
cam_image30.jpg,960,540,ten,202,47,456,344
IMG_2515.JPG,378,504,jack,56,172,226,369
cam_image3.jpg,960,540,king,326,99,473,291
cam_image3.jpg,960,540,king,496,88,658,277
cam_image3.jpg,960,540,king,501,291,674,513
cam_image3.jpg,960,540,king,320,295,481,516
cam_image33.jpg,960,540,nine,281,112,504,386
IMG_2603.JPG,378,504,king,209,208,372,410


Open `generate_tfrecord.py` and:
1. replace `import tensorflow` with `import tensorflow.compat.v1`.
2. Modify `class_test_to_int()` function with label-id mapping.

In [3]:
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record
print("")
!ls -lh | grep .*\.record$

Successfully created the TFRecords: /models/research/object_detection/train.record
Successfully created the TFRecords: /models/research/object_detection/test.record

-rw-r--r-- 1 root root 6.7M Nov  3 18:40 test.record
-rw-r--r-- 1 root root  30M Nov  3 18:40 train.record


### Training

Create label map and save it in `training/labelmap.pbtxt` with the same mapping as in `generate_tfrecord.py`.

Get model checkpoint

In [4]:
!wget "http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz"
!tar xvzf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

--2019-11-03 18:40:44--  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.28.112, 2800:3f0:4005:401::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.28.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149119618 (142M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’

faster_rcnn_incepti 100%[===================>] 142.21M  1.89MB/s    in 79s     

2019-11-03 18:42:04 (1.79 MB/s) - ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [149119618/149119618]

faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_v2_coco_2018_01_28/mo

**Don't run for cards example**
1. Copy config file from `./samples/configs/` to `./training/`
2. Adapt parameters in config file:
    - `num_classes`
    - `fine_tune_checkpoint`
    - train_input_reader: `input_path`, `label_map_path`
    - `num_examples`: images/test/
    - eval_input_reader: `input_path`, `label_map_path`

In [5]:
!cp samples/configs/faster_rcnn_inception_v2_pets.config training/

Move train.py from `legacy/` tom `.`

In [6]:
!cp legacy/train.py .

Train model (needs to be stopped manually)

In [9]:
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






Instructions for updating:
Use object_detection/model_main.py.
W1103 19:00:40.089211 140338541487936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.

W1103 19:00:40.089445 140338541487936 module_wrapper.py:139] From train.py:90: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1103 19:00:40.089964 140338541487936 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:102: The name tf.gfile.G

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
W1103 19:00:40.960961 140338541487936 deprecation.py:323] From /models/research/object_detection/core/batcher.py:101: batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1103 19:00:40.965812 140338541487936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/input.py:752: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.d


W1103 19:00:44.210960 140338541487936 module_wrapper.py:139] From /models/research/object_detection/meta_architectures/faster_rcnn_meta_arch.py:189: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

Instructions for updating:
Use keras.layers.flatten instead.
W1103 19:00:44.870241 140338541487936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1103 19:00:44.873371 140338541487936 regularizers.py:98] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1103 19:00:44.897302 140338541487936 regularizers.py:98] Scale of 0 disables regularizer.

W1103 19:00:45.201364 140338541487936 module_wrapper.py:139] From /models/research/object_detection/c

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W1103 19:00:51.828465 140338541487936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/slim/python/slim/learning.py:742: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2019-11-03 19:00:52.533533: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-03 19:00:52.543822: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2773865000 Hz
2019-11-03 19:00:52.544231: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xcaa4740 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-11-03 19:00:52.544297: I tensorflow/compiler/xla/service/service.cc:176]   StreamE

INFO:tensorflow:global step 33: loss = 2.8120 (1.721 sec/step)
I1103 19:02:12.777174 140338541487936 learning.py:507] global step 33: loss = 2.8120 (1.721 sec/step)
INFO:tensorflow:global step 34: loss = 1.0942 (1.721 sec/step)
I1103 19:02:14.499249 140338541487936 learning.py:507] global step 34: loss = 1.0942 (1.721 sec/step)
INFO:tensorflow:global step 35: loss = 2.3436 (1.714 sec/step)
I1103 19:02:16.213970 140338541487936 learning.py:507] global step 35: loss = 2.3436 (1.714 sec/step)
INFO:tensorflow:global step 36: loss = 1.0571 (1.714 sec/step)
I1103 19:02:17.928613 140338541487936 learning.py:507] global step 36: loss = 1.0571 (1.714 sec/step)
INFO:tensorflow:global step 37: loss = 2.1739 (1.707 sec/step)
I1103 19:02:19.636411 140338541487936 learning.py:507] global step 37: loss = 2.1739 (1.707 sec/step)
INFO:tensorflow:global step 38: loss = 2.8135 (1.759 sec/step)
I1103 19:02:21.396193 140338541487936 learning.py:507] global step 38: loss = 2.8135 (1.759 sec/step)
INFO:tenso

...

INFO:tensorflow:global step 6109: loss = 0.1142 (1.692 sec/step)
I1103 22:00:35.484159 140338541487936 learning.py:507] global step 6109: loss = 0.1142 (1.692 sec/step)
INFO:tensorflow:global step 6110: loss = 0.1236 (1.716 sec/step)
I1103 22:00:37.201257 140338541487936 learning.py:507] global step 6110: loss = 0.1236 (1.716 sec/step)
INFO:tensorflow:global step 6111: loss = 0.1442 (1.673 sec/step)
I1103 22:00:38.875355 140338541487936 learning.py:507] global step 6111: loss = 0.1442 (1.673 sec/step)
INFO:tensorflow:global step 6112: loss = 0.1704 (1.673 sec/step)
I1103 22:00:40.549875 140338541487936 learning.py:507] global step 6112: loss = 0.1704 (1.673 sec/step)
INFO:tensorflow:global step 6113: loss = 0.0889 (1.678 sec/step)
I1103 22:00:42.229288 140338541487936 learning.py:507] global step 6113: loss = 0.0889 (1.678 sec/step)
INFO:tensorflow:global step 6114: loss = 0.4110 (1.676 sec/step)
I1103 22:00:43.906540 140338541487936 learning.py:507] global step 6114: loss = 0.4110 (1.

INFO:tensorflow:global step 6156: loss = 0.0898 (1.680 sec/step)
I1103 22:01:56.995320 140338541487936 learning.py:507] global step 6156: loss = 0.0898 (1.680 sec/step)
INFO:tensorflow:global step 6157: loss = 0.1263 (1.674 sec/step)
I1103 22:01:58.670262 140338541487936 learning.py:507] global step 6157: loss = 0.1263 (1.674 sec/step)
INFO:tensorflow:global step 6158: loss = 0.1666 (1.924 sec/step)
I1103 22:02:00.594701 140338541487936 learning.py:507] global step 6158: loss = 0.1666 (1.924 sec/step)
INFO:tensorflow:global step 6159: loss = 0.0661 (1.655 sec/step)
I1103 22:02:02.251082 140338541487936 learning.py:507] global step 6159: loss = 0.0661 (1.655 sec/step)
INFO:tensorflow:global step 6160: loss = 0.2442 (1.915 sec/step)
I1103 22:02:04.167254 140338541487936 learning.py:507] global step 6160: loss = 0.2442 (1.915 sec/step)
INFO:tensorflow:global step 6161: loss = 0.2549 (1.672 sec/step)
I1103 22:02:05.839960 140338541487936 learning.py:507] global step 6161: loss = 0.2549 (1.

INFO:tensorflow:global step 6204: loss = 0.0822 (1.737 sec/step)
I1103 22:03:19.604190 140338541487936 learning.py:507] global step 6204: loss = 0.0822 (1.737 sec/step)
INFO:tensorflow:global step 6205: loss = 0.1307 (2.018 sec/step)
I1103 22:03:21.623749 140338541487936 learning.py:507] global step 6205: loss = 0.1307 (2.018 sec/step)
INFO:tensorflow:global step 6206: loss = 0.0542 (1.669 sec/step)
I1103 22:03:23.293556 140338541487936 learning.py:507] global step 6206: loss = 0.0542 (1.669 sec/step)
INFO:tensorflow:global step 6207: loss = 0.0845 (1.683 sec/step)
I1103 22:03:24.977855 140338541487936 learning.py:507] global step 6207: loss = 0.0845 (1.683 sec/step)
INFO:tensorflow:global step 6208: loss = 0.0738 (1.927 sec/step)
I1103 22:03:26.906049 140338541487936 learning.py:507] global step 6208: loss = 0.0738 (1.927 sec/step)
INFO:tensorflow:global step 6209: loss = 0.0692 (1.663 sec/step)
I1103 22:03:28.570350 140338541487936 learning.py:507] global step 6209: loss = 0.0692 (1.

INFO:tensorflow:global step 6253: loss = 0.0651 (2.368 sec/step)
I1103 22:04:45.183064 140338541487936 learning.py:507] global step 6253: loss = 0.0651 (2.368 sec/step)
INFO:tensorflow:global step 6254: loss = 0.1801 (1.966 sec/step)
I1103 22:04:47.150193 140338541487936 learning.py:507] global step 6254: loss = 0.1801 (1.966 sec/step)
INFO:tensorflow:global step 6255: loss = 0.1430 (1.950 sec/step)
I1103 22:04:49.101213 140338541487936 learning.py:507] global step 6255: loss = 0.1430 (1.950 sec/step)
INFO:tensorflow:global step 6256: loss = 0.0794 (1.884 sec/step)
I1103 22:04:50.986596 140338541487936 learning.py:507] global step 6256: loss = 0.0794 (1.884 sec/step)
INFO:tensorflow:global step 6257: loss = 0.0350 (2.096 sec/step)
I1103 22:04:53.084698 140338541487936 learning.py:507] global step 6257: loss = 0.0350 (2.096 sec/step)
INFO:tensorflow:global step 6258: loss = 0.1555 (1.774 sec/step)
I1103 22:04:54.859589 140338541487936 learning.py:507] global step 6258: loss = 0.1555 (1.

INFO:tensorflow:global step 6301: loss = 0.2524 (1.828 sec/step)
I1103 22:06:15.640875 140338541487936 learning.py:507] global step 6301: loss = 0.2524 (1.828 sec/step)
INFO:tensorflow:global step 6302: loss = 0.0707 (2.381 sec/step)
I1103 22:06:18.023354 140338541487936 learning.py:507] global step 6302: loss = 0.0707 (2.381 sec/step)
INFO:tensorflow:global step 6303: loss = 0.2216 (2.338 sec/step)
I1103 22:06:20.362655 140338541487936 learning.py:507] global step 6303: loss = 0.2216 (2.338 sec/step)
INFO:tensorflow:global step 6304: loss = 0.2749 (1.735 sec/step)
I1103 22:06:22.098947 140338541487936 learning.py:507] global step 6304: loss = 0.2749 (1.735 sec/step)
INFO:tensorflow:global step 6305: loss = 0.0546 (1.775 sec/step)
I1103 22:06:23.875399 140338541487936 learning.py:507] global step 6305: loss = 0.0546 (1.775 sec/step)
INFO:tensorflow:global step 6306: loss = 0.0925 (1.762 sec/step)
I1103 22:06:25.639066 140338541487936 learning.py:507] global step 6306: loss = 0.0925 (1.

Export model

In [10]:
!ls -l training/

total 596892
-rw-r--r-- 1 root root       271 Nov  3 22:01 checkpoint
-rw-r--r-- 1 root root  57495546 Nov  3 22:07 events.out.tfevents.1572807661.67d292914e56
-rw-r--r-- 1 root root      3889 Nov  3 19:00 faster_rcnn_inception_v2_pets.config
-rw-r--r-- 1 root root      7789 Nov  3 18:56 faster_rcnn_inception_v2_pets.config.bak
-rw-r--r-- 1 root root  10460904 Nov  3 19:01 graph.pbtxt
-rw-r--r-- 1 root root       225 Oct 13 02:15 labelmap.pbtxt
-rw-r--r-- 1 root root 103173376 Nov  3 21:21 model.ckpt-4756.data-00000-of-00001
-rw-r--r-- 1 root root     25563 Nov  3 21:21 model.ckpt-4756.index
-rw-r--r-- 1 root root   5439659 Nov  3 21:21 model.ckpt-4756.meta
-rw-r--r-- 1 root root 103173376 Nov  3 21:31 model.ckpt-5102.data-00000-of-00001
-rw-r--r-- 1 root root     25563 Nov  3 21:31 model.ckpt-5102.index
-rw-r--r-- 1 root root   5439659 Nov  3 21:31 model.ckpt-5102.meta
-rw-r--r-- 1 root root 103173376 Nov  3 21:41 model.ckpt-5444.data-00000-of-00001
-rw-r--r-- 1 root root     25563 No

In [11]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-6124 --output_directory inference_graph

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1103 22:08:45.919301 139990602536768 module_wrapper.py:139] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1103 22:08:45.929482 139990602536768 module_wrapper.py:139] From /models/research/object_detection/exporter.py:389: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1103 22:08:45.929881 139990602536768 module_wrapper.py:139] From /models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W1103 22:08:45.988798 139990602536768 module_wrappe


W1103 22:08:50.488631 139990602536768 module_wrapper.py:139] From /models/research/object_detection/exporter.py:268: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W1103 22:08:50.489014 139990602536768 deprecation.py:323] From /models/research/object_detection/exporter.py:370: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step

W1103 22:08:50.493596 139990602536768 module_wrapper.py:139] From /models/research/object_detection/exporter.py:402: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README

242 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz


W1103 22:08:52.017541 139990602536768 module_wrapper.py:139] From /models/research/object_detection/exporter.py:419: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.


W1103 22:08:53.136866 139990602536768 module_wrapper.py:139] From /models/research/object_detection/exporter.py:332: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-11-03 22:08:53.137474: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2019-11-03 22:08:53.137540: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2019-11-03 22:08:53.137573: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (67d292914e56): /proc/driver/nvidia/version does not exist
2019-11-03 22:08:53.137859: I tensorflow/core/platform


W1103 22:08:58.597692 139990602536768 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:188: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Writing pipeline config file to inference_graph/pipeline.config
I1103 22:08:58.598068 139990602536768 config_util.py:190] Writing pipeline config file to inference_graph/pipeline.config


In [13]:
!ls -lh inference_graph/

total 102M
-rw-r--r-- 1 root root   77 Nov  3 22:08 checkpoint
-rw-r--r-- 1 root root  51M Nov  3 22:08 frozen_inference_graph.pb
-rw-r--r-- 1 root root  50M Nov  3 22:08 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  16K Nov  3 22:08 model.ckpt.index
-rw-r--r-- 1 root root 1.9M Nov  3 22:08 model.ckpt.meta
-rw-r--r-- 1 root root 3.3K Nov  3 22:08 pipeline.config
-rw-r--r-- 1 root root  163 Oct 13 02:15 placeholder.txt
drwxr-xr-x 3 root root 4.0K Nov  3 22:08 saved_model


### Perform inference

In [18]:
!pwd

/models/research/object_detection


In [21]:
!cat Object_detection_image.py

In NoneType related error, reinstall opencv with `apt install opencv-python`.

To `Object_detection_image.py`, add before last line `cv2.imwrite('/path/to/save/boxed_image.JPG', image)` and comment last line out.

In [36]:
!python Object_detection_image.py




2019-11-03 23:21:37.561608: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2019-11-03 23:21:37.561679: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2019-11-03 23:21:37.561708: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (67d292914e56): /proc/driver/nvidia/version does not exist
2019-11-03 23:21:37.562080: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-03 23:21:37.568567: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2773865000 Hz
2019-11-03 23:21:37.569660: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x622a430 initialized for platform Host (this does not guarantee that 